#PyTorch Intro - Trenowanie i Ewaluacja Sieci Neuronowej - Wykład
Notatnik demonstruje implementację, trenowanie i ewaluację klasyfikatora opartego o sieć neuronowną o architekturze wielowarstwowego perceptronu (MLP - *multi-layer perceptron*).

W notatniku wykorzystano następujące narzędzia:
-  **PyTorch** - biblioteka głębokiego uczenia (tworzenie, trenowanie i ewaluacja głębokich sieci neuronowych). [link](https://pytorch.org/)
-   **PyTorch Lightning** - biblioteka upraszczająca implementację pętli trenowania i ewaluacji modeli w PyTorch. [link](https://lightning.ai/docs/pytorch/stable/)
-  Mobuł **Datasets** z biblioteki **HuggingFacce** - dostęp do wielu zbiorów danych z różnych dziedzin (audio, wizja komputerowa, przetwarzanie języka naturalnego). [link](https://huggingface.co/docs/datasets/index)
-   **W&B (*Weights and Biases*) Models** - serwis w chmurze do zarządzania eksperymentami, monitorowania przebiegu treningu i logowania wyników ewaluacji modeli. [link](https://wandb.ai/site)
-   **TorchMetrics** - biblioteka ponad 100 metryk do ewaluacji różnych typów modeli uczenia maszynowego. [link](https://lightning.ai/docs/torchmetrics/stable/)

##Przygotowanie środowiska
Upewnij się, że notatnik jest uruchomiony na maszynie z GPU. Jesli GPU nie jest dostępne zmień typ maszyny (Runtime | Change runtime type) i wybierz T4 GPU.

In [ ]:
!nvidia-smi

Instalacja dodatkowych bibliotek: datasets (z biblioteki HuggingFace), TorchMetrics i W&B (Weights and Biases) Models.

In [ ]:
!pip install -q datasets
!pip install -q torchmetrics
!pip install -q wandb

Import bibliotek.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

print(f"Wersja biblioteki PyTorch: {torch.__version__}")

Sprawdzenie dostępności GPU.

In [ ]:
print(f"Dostępność GPU: {torch.cuda.is_available()}")
print(f"Typ GPU: {torch.cuda.get_device_name(0)}")

#Klasyfikacja sentymentu tekstu w języku naturalnym
Problemem rozwiązywanym w notatniku jest **klasyfikacja sentymentu**, czyli określenie wydźwięku emocjonalnego tekstu w języku naturalnym.
Dla podanego tekstu w języku angielskim należy określić czy ma on wydźwięk negatywny czy pozytywny (klasyfikacja dwuklasowa).

W notatniku problem klasyfikacji sentymentu rozwiążemy bez wykorzystania wielkich modeli językowych.
Zastosujemy tradycyjne podejście oparte o reprezentację typu **worek słów (*bag-of-words*)**  w którym analizowany tekst reprezentowany jest jako nieuporządkowana kolekcja (worek) słów. Pozycja słowa w tekście nie ma znaczenia, uwzględniana jest tylko liczba jego wystąpień. Więcej informacji znajdziesz tutaj: [link](https://en.wikipedia.org/wiki/Bag-of-words_model).




##Pobranie danych
Wykorzystamy zbiór danych Yelp Polarity zawierający prawie 600 tysięcy opinii z serwisu Yelp wraz z numeryczną etykietą określającą sentyment wypowiedzi (0: negatywny i 1:pozytywny).
Biblioteka HuggingFace (moduł datasets) udostępnia setki zbiorów danych z różnych dziedzin (audio, wizja komputerowa, przetwarzanie języka naturalnego). Więcej informacji: [link](https://huggingface.co/docs/datasets/index).

Polecenie `load_dataset` pobiera zbiór danych złożony z części treningowej (`dataset["train"]`) i testowej (`dataset["test"]`).
Zgodnie z dobrą praktyką z części treningowej wydzielimy dodatkową część walidacyjną.
Z uwagi na ograniczenia sprzętowe platformy COLAB rozmiar każdej części zbioru danych (treningowej, walidacyjnej i testowej) zostanie ograniczony.

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("fancyzhx/yelp_polarity")
print(dataset)

test_dataset = dataset["test"]

# Wydziel część walidacyjną ze zbioru treningowego
split_dataset = dataset["train"].train_test_split(test_size=0.2)  # 80% train, 20% validation
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

# Utwórz mniejsze podzbiory treningowe, walidacyjne i testowe
train_dataset = train_dataset.shuffle().select(range(int(len(train_dataset) * .15)))
val_dataset = val_dataset.shuffle().select(range(int(len(val_dataset) * .1)))
test_dataset = test_dataset.shuffle().select(range(int(len(test_dataset) * .4)))

print(f"Liczba próbek w zbiorze treningowym: {len(train_dataset)}")
print(f"Liczba próbek w zbiorze walidacyjnym: {len(val_dataset)}")
print(f"Liczba próbek w zbiorze testowym: {len(test_dataset)}")

In [ ]:
sample = train_dataset[123]
print(f"\nPrzykładowy element ze zbioru danych:")
print(f"{sample['text']=}")
print(f"{sample['label']=}")

Sprawdzenie liczby elementów z każdej klasy w zbiorze treningowym.

In [ ]:
# Count occurrences of each label
unique_labels, counts = np.unique(train_dataset['label'], return_counts=True)

# Plot histogram
plt.bar(unique_labels, counts, color=['skyblue', 'orange'], edgecolor='black')
plt.title('Histogram of Binary Labels')
plt.xlabel('Label')
plt.ylabel('Frequency')
plt.xticks(unique_labels)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Ekstrakcja cech
Do ekstrakcji cech z tekstu wykorzystamy **metodę TF-IDF** (*term frequency-inverse document frequency*) opartą o podejście typu worek słów (*bag-of-words*). TF-IDF wyznacza miarę istotności (wagę) słów w dokumencie (tekście) będącym częścią większego zbioru dokumentów (korpusu tekstowego). TF-IDF jest obliczane dla każdego słowa jako iloczyn dwóch wartości:
$$\textit{tf-idf} = \textit{tf} \cdot \textit{idf}$$
gdzie $\textit{tf}\ $ jest czynnikiem zależnym od względnej częstości słowa w dokumencie, a $\textit{idf}\ $ jest tym mniejsze im w większej liczbie dokumentów występuje słowo, np.:
$$\textit{idf} = \log \frac{1+n}{1+\textit{df}(t)} + 1 \, ,$$
gdzie $n$ jest liczbą wszystkich dokumentów a $\textit{df}(t)$ liczbą dokumentów w których występuje słowo $t$.
Intuicja jest taka, że bardzo często występujące słowa (np. *a*, *the*, *and*) nie są informatywne.
Więcej informacji: [link](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Funkcja `TfidfVectorizer` z biblioteki `scikit-learn` tworzy rzadką macierz cech TF-IDF na podstawie zbioru dokumentów testowych. Każdy wiersz macierzy reprezentuje jeden dokument ze zbioru a kolumny zawierają wagi każdego słowa wyznaczone metodą TF-IDF.
Aby zmniejszyć rozmiar wynikowej macierzy wykorzystamy parametr `max_features` aby ograniczyć liczbę słów (i co za tym idzie kolumn macierzy) do 10 tysięcy najczęściej występujących.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tqdm import tqdm

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vocab_size = 10000

vectorizer = TfidfVectorizer(
    max_features=vocab_size,    # Ogranicz do max_features najczęściej występujących słów
    lowercase=True,             # Przekształć na małe litery
    analyzer='word',            # Analiza na poziomie słów (a nie pojedynczych znaków)
    ngram_range=(1, 1),         # Unigramy (pojedynczne słowa)
    stop_words="english"        # Usuń częste słowa w języku angielskim (np. a, the, and)
)

# Ekstrakcja cech
# Na zbiorze treningowym stosujemy funkcję fit_transform() która wyznacza cech i dokonuje ich ekstrakcji
train_tfidf_features = vectorizer.fit_transform(train_dataset["text"])
# Na zbiorze walidacyjnym i testowym stosujemy funkcję transform() które dokonuje ekstrakcji tych samych cech co na zbiorze treningowym
val_tfidf_features = vectorizer.transform(val_dataset["text"])
test_tfidf_features = vectorizer.transform(test_dataset["text"])

In [ ]:
print(f"Rozmiar macierzy TF-IDF dla zbioru treningowego: {train_tfidf_features.shape}")
print(f"{train_tfidf_features.dtype=}\n")

feature_names = vectorizer.get_feature_names_out()
print(f"Liczba cech: {len(feature_names)}")
print(f"Przykładowe cechy: {np.random.choice(feature_names, 20)}\n")

Wyświetlenie wektora cech dla wybranego elementu zbioru treningowego.

In [ ]:
ndx = 111
print(f"Element zbioru treningowego o indeksie={ndx}")
print(f"{train_dataset['text'][ndx]=}\n")
features = train_tfidf_features[ndx]
print(f"Cechy elementu o indeksie={ndx}:")
non_zero_cols = features.nonzero()[1]

non_zero_cols = sorted(non_zero_cols)
for i in non_zero_cols:
    print(f"Kolumna: {i} ({feature_names[i]})   Waga: {features[0, i]:.5f}")

Histogram liczby niezerowych cech elementów zbioru treningowego.
Jak widać, macierz cech `train_tfidf_features` jest bardzo rzadka - średnio niecałe 50 cech (kolumn macierzy `train_tfidf_features`) z 10 tysięcy jest niezerowe.

In [ ]:
# train_tfidf_features jest rzadką macierzą w formacie CSR (compressed sparse row matrix)
# Poniższy trik pozwala szybko wyznaczyć liczbę niezerowych elementów w każdym wierszu
# .indptr jest N+1 elementową tablicą taką, że indptr[i+1]-indptr[i] jest liczbą niezerowych wartości w i-tym wierszu
# Patrz: https://stackoverflow.com/questions/52299420/scipy-csr-matrix-understand-indptr

non_zero_counts = np.diff(train_tfidf_features.indptr)
print(f"Średnia liczba niezerowych cech w próbkach: {non_zero_counts.mean():.2f}")

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(non_zero_counts, bins=64)
plt.title('Histogram liczby niezerowych cech w próbce')
plt.xlabel('Liczba niezerowych cech')
plt.ylabel('Liczba próbek')
plt.grid(axis='y', linestyle='--', alpha=0.7)

##Przygotowanie zbioru danych w PyTorch

Biblioteka PyTorch implementuje mechanizmy umożliwiające dostęp do danych różnego typu (audio, obrazowe, tekstowe,...) w ustandaryzowany sposób. Do tego celu wykorzystywane są dwa rodzaje obiektów:
*   **Zbiory danych (*Datasets*)** - umożliwiają dostęp do pojedyncznych elementów zbioru danych. Klasy implementujące dostęp do zbioru danych powinny być pochodnymi abstrakcyjnej klasy `torch.utils.data.Dataset`. Wyróżniamy dwa rodzaje zbiorów danych: mapowane (*map-style*) i iteracyjne (*iterable-style*).
Najczęściej wykorzystywane mapowane zbiory danych umożliwiają dostęp do zindeksowanych elementów zbioru. Muszą implementować metody `__getitem__(ndx)` zwracającą element (np. sekwencję tekstową lub obraz wraz z etykietą) o indeksie `ndx` i `__len__()` zwracającą liczbę elementów zbioru danych.

*   **Ładowarka danych (DataLoader)** -  klasa `torch.utils.data.DataLoader` pozwala opakować dostęp do zbioru danych i utworzyć iterator zwracający wsady treningowe złożone z wielu elementów. Pozwala wykorzystać wiele współbieżnych procesów roboczych w celu efektywnej budowy dużych wsadów treningowych.

In [ ]:
import torch

from torch.utils.data import TensorDataset, DataLoader

Aby przygotować dane do trenowania modelu w PyTorch należy:
1.   Zaimplementować klasę definiującą dostęp do zbioru danych - dziedziczącą z `torch.utils.data.Dataset` z metodami `__getitem__(ndx)` i `__len__()`.
Metoda `__getitem__(ndx)` powinna zwracać element zbioru danych o indeksie `ndx` wraz z etykietą.
Alternatywnie można wykorzystać zaimplementowaną w PyTorch klasę definiującą dostęp do publicznie dostępnego zbioru danych, patrz: [link](https://pytorch.org/vision/stable/datasets.html).
2.   Dla każdej części zbioru danych (treningowej, walidacyjnej i testowej) utworzyć obiekt zaimplementowanej klasy.
3.   Dla każdej części zbioru danych utworzyć ładowarkę danych klasy `torch.utils.data.DataLoader` zwracającą wsady treningowe złożone z wielu elementów.

W naszym przypadku, elementy zbioru danych są już wczytane do pamięci.
Dla części treningowej cechy są zapisane w rzadkiej macierzy `train_tfidf_features` a etykiety w `train_dataset['label']`.
Zamiast implementować własną klasę definiującą dostęp do zbioru danych przekształcimy zbiór danych do postaci tensorów i wykorzystamy klasę `torch.utils.data.TensorDataset`, dziedziczącą z `torch.utils.data.Dataset`, która tworzy obiekt zbioru danych na podstawie podanych tensorów.

In [ ]:
def make_dataset(sparse_features, labels):
    # Zamień rzadką macierz cech na zwykłą (gęstą) macierz ndarray
    dense_features = sparse_features.astype(np.float32).todense()
    # Utwórz zbiór danych na podstawie tensora z cechami i tensora z etykietami
    dataset = TensorDataset(
        torch.from_numpy(dense_features),
        torch.tensor(labels, dtype=torch.int64)
    )
    return dataset

# Utwórz trzy zbiory danych: treningowy, walidacyjny i testowy
datasets = {
    'train': make_dataset(train_tfidf_features, train_dataset['label']),
    'val': make_dataset(val_tfidf_features, val_dataset['label']),
    'test': make_dataset(test_tfidf_features, test_dataset['label'])
}

# Wyświetl przykładowy element
print(datasets['train'][0])

Utworzenie ładowarek danych dla każdej części zbioru danych (treningowej, walidacyjnej i testowej). Przydatne parametry:
- `batch_size` - rozmiar wsadu treningowego.
- `shuffle` - czy wybierać elementy zbioru danych w losowej kolejności. Powinien być ustawiony na `True` tylko dla zbioru treningowego.
- `num_workers` - liczba procesów roboczych ładowarki danych. Na COLAB zalecane jest ustawienie 0 (ładowarka danych działa w głównym procesie). Na innych platformach należy ustawić większe wartości, zależne od dostępnych zasobów obliczeniowych.

In [ ]:
batch_size = 256

dataloaders = {split: DataLoader(datasets[split], batch_size=batch_size, shuffle=split=='train', num_workers=0) for split in datasets}

Obiekt klasy `DataLoader` jest iteratorem generującym kolejne wsady danych. W naszym przypadku wsad złożony jest z dwóch części: tensora cech o rozmiarach `(batch_size=256, 10000)` i tensora etykiet o rozmiarze `(batch_size=256,)`.
W ogólności wsady mogą mieć bardziej złożoną strukturę, np. słowniki których wartości są tensorami.


In [ ]:
for X_batch, y_batch in dataloaders['train']:
    print(f"{X_batch.shape=}")
    print(f"{y_batch.shape=}")
    break

## Utworzenie modelu sieci neuronowej

Modele sieci neuronowych implementujemy w PyTorch jako klasy dziedziczące z klasy `torch.nn.Module`. W metodzie `__init__(...)` tworzone są składowe architektury modelu, takie jak warstwy tworzące sieć neuronową.
W metodzie `forward(...)` zaimplementowana jest logika przetwarzania wejściowego tensora (bądź tensorów) przez sieć. Wywoływane są kolejne warstwy sieci z odpowiednimi argumentami i zwracany jest wynikowy tensor.

W notatniku zaimplementujemy klasyfikator jako perceptron wielowarstwowy złożony z kilku warstw liniowych `torch.nn.Linear` oddzielonych warstwami nieliniowymi `torch.nn.ReLU`.

**WAŻNE:** Klasa definiująca sieć neuronową bądź moduł sieci neuronowej musi być pochodną klasy `torch.nn.Module`.
Architekturę modelu rozbijemy na dwie części:
- **Blok ekstrakcji cech** (`self.feature_extractor`). Wykorzystamy klasę `torch.nn.Sequential` aby połączyć dwie warstwy liniowe i następujące po nich nieliniowości w jeden sekwencyjny blok. Zauważmy, że rozmiar wejściowy pierwszej warstwy sekwencyjnej jest równy rozmiarowi wektora cech z bazy danych (`self.vocab_size`)
- **Końcowej warstwy liniowego klasyfikatora** (`self.linear`). Rozmiar wyjściowy liniowego klasyfikatora jest równy liczbie klas (`n_classes=2` - sentyment negatywny lub pozytywny).
Aby zmniejszyć ryzyko przeuczenia i poprawić generalizację sieci przed liniowym klasyfikatorem zastosujemy warstwę odrzutu `nn.Dropout`.

In [ ]:
import torch.nn as nn


class SimpleNet(nn.Module):
    def __init__(self, vocab_size: int, n_classes: int):
        super().__init__()
        self.vocab_size = vocab_size
        self.n_classes = n_classes

        self.feature_extractor = nn.Sequential(
            nn.Linear(self.vocab_size, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU()
        )
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(16, n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.feature_extractor(x)
        features = self.dropout(features)
        logits = self.linear(features)
        return logits

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print("Urządzenie: {}".format(device))

In [ ]:
# Utwórz model sieci neuronowej i prześlij na właściwe urządzenie
classifier = SimpleNet(vocab_size, n_classes=2)
classifier.to(device)

print(classifier)

**Uwaga:** Aby przesłać moduł sieci neuronowej (obiekt klasy dziedziczącej z `nn.Module`) na urządzenie wystarczy polecenie:
```
model.to(device)
```
Aby przesłać tensor x na urządzenie należy napisać:
```
x = x.to(device)
```
Metoda `to(device)` dla tensora tworzy jego kopię na podanym urządzeniu (i zwraca referencję do kopii).

Sprawdzenie działania sieci na losowych danych.


In [ ]:
# Utworzenie 5-elementowego wsadu tensorów o vocab_size elementach każdy
x = torch.rand((5, vocab_size))
print(x)
print(f"{x.shape=}\n")

x = x.to(device)        # Przenieś tensor na GPU

logits = classifier(x)
print(f"{logits=}")
print(f"{logits.shape=}\n")

probabilities = nn.functional.softmax(logits, dim=-1)
print(f"{probabilities=}")

Zauważmy, że dla każdego elementu zbioru danych sieć zwraca wektor dwóch wartości rzeczywistych z zakresu od minus do plus nieskończoności.
 Aby uzyskać rozkład prawdopodobieństwa klas należy zastosować funkcję `nn.functional.softmax` na wyjściu z modułu sieci zdefiniowaną jako:

 $$\sigma(\mathbf{z})_i = \frac{exp{(z_i)}}{\sum_j{\exp(z_j)}} \, ,$$
 gdzie $\mathbf{z} =(z_{1},\dotsc ,z_{K})\in \mathbb {R} ^{K}$ jest wektorem nieznormalizowanych wyjść z sieci.

**WAŻNE:**
Zwykle klasyfikatory oparte o sieci neuronowe zwracają wartości zwane **logitami** które możemy interpretować jako nieznormalizowane logarytmy prawdopodobieństwa.
Ze względów numerycznych podczas trenowania modelu **funkcję straty należy wyznaczać na podstawie logitów**, a NIE w oparciu o prawdopodobieństwa klas.
Z tego względu NIE należy stosować operacji softmax wewnątrz modułu sieci neuronowej. Moduł sieci powinien zwracać nieznormalizowane wartości (logity).

##Pętla treningowa

Zaimplementowana poniżej funkcja `train` zawiera typową pętlę treningową modelu sieci neuronowej.

*   Trening modelu jest podzielony na **epoki**. W czasie jednej epoki następuje jednokrotne przejście przez cały zbiór danych.
*   Każda epoka podzielona jest na **dwie fazy** - fazę **treningu** (train) i **walidacji** (val).
*   W każdej fazie iteracyjnie pobierane są kolejne **wsady** ze zbioru danych (treningowego albo walidacyjnego). Dla każdego wsadu wyznaczane są wartości funkcji straty (entropia krzyżowa) i metryki jakości modelu (dokładność). W fazie treningu dodatkowo wyznaczane są gradienty funkcji straty względem parametrów sieci (przejście w tył - `loss.backward()`) i optymalizowane są parametry sieci (`optimizer.step()`).

**WAŻNE - specyfika PyTorcha:**

*   Moduł sieci neuronowej może być w trybie treningowym (`train`) lub ewaluacyjnym (`eval`). W fazie trenowania model musi zostać przełączony w tryb treningowy poleceniem `model.train()`. W pozostałych fazach model **musi** zostać przełączony w tryb ewaluacji
poleceniem `model.eval()`.
Jeśli chcemy wykorzystać wytrenowany model do wnioskowania **należy pamiętać o przełączeniu go w tryb ewaluacji**, w przeciwnym przypadku wyniki działania sieci mogą być błędne.
Różnice między trybem `train` i `eval`: [link](https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch).
*   Domyślnie gradienty zapamiętywane w tensorach podlegających optymalizacji (wagach sieci) akumulują się po każdym wywołaniu przejścia w tył (metody `backward`).Dlatego konieczne jest wyzerowanie gradientów poleceniem `optimizer.zero_grad()`. Więcej informacji: [link](https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch).
*   Wywołanie modułu sieci (klasy dziedziczącej z `nn.Module`) i wykonanie przejścia w przód wykonujemy poleceniem `model(X_batch)` a NIE `model.forward(X_batch)`. Nie należy bezpośrednio wywowyłać metody `forward`!



Oprócz biblioteki PyTorch wykorzystamy następujące narzędzia:
-   **W&B (*Weights and Biases*) Models** - serwis w chmurze do zarządzania eksperymentami, monitorowania przebiegu treningu i logowania wyników ewaluacji modeli. [link](https://wandb.ai/site)
-   **TorchMetrics** - biblioteka ponad 100 metryk do ewaluacji różnych typów modeli uczenia maszynowego. [link](https://lightning.ai/docs/torchmetrics/stable/)

In [ ]:
import torchmetrics


def train(model: nn.Module, loaders: dict[DataLoader], criterion: nn.Module,
          optimizer: torch.optim.Optimizer, lr_scheduler, num_epochs: int):

    # Metryki wyznaczane dla wsadu
    metric_loss = torchmetrics.aggregation.MeanMetric().to(device)
    metric_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2).to(device)

    # Run all epochs
    for epoch in range(1, num_epochs+1):

        for phase in ['train', 'val']:
            # WAŻNE - przełącz model w odpowiedni tryb
            if phase == 'train':
                model.train()  # Przełącz model w tryb treningowy
            else:
                model.eval()   # Przełącz model w tryb ewaluacyjny

            # Loop over each batch in the data loader
            for X_batch, target in tqdm(loaders[phase]):
                # Przenieś dane na odpowiednie urządzenie
                X_batch, target = X_batch.to(device), target.to(device)

                # Wyzeruj gradienty parametrów sieci
                # BARDZO WAŻNY KROK - domyślnie gradienty nie są zerowane i akumulują się dla wielu kroków przejścia w tył
                optimizer.zero_grad()

                # Przejście w przód (forward)
                # Śledzenie historii obliczeń tylko w fazie trenowania
                with torch.set_grad_enabled(phase == 'train'):
                    logits = model(X_batch)
                    _, preds = torch.max(logits, dim=1)
                    loss = criterion(logits, target)

                    # Zaktualizuj wartości metryk
                    metric_loss(loss)
                    metric_acc(preds, target)

                    # Przejście w tył (backward) i krok optymalizacji parametrów sieci tylko w fazie trenowania
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

            # Wyznacz średnie wartości metryk dla wsadu
            acc = metric_acc.compute()
            mean_loss = metric_loss.compute()
            current_lr = lr_scheduler.get_last_lr()[0]
            print(f"(Epoch {epoch}/[{phase}]) Loss:\t{mean_loss:.3f}   Accuracy: {acc:.3f}   lr: {current_lr}")
            metrics = {
                f"{phase}/loss": mean_loss,
                f"{phase}/accuracy": acc,
                f"{phase}/lr": current_lr,
            }
            wandb.log(metrics, step=epoch)

            # Wyzeruj metryki
            metric_loss.reset()
            metric_acc.reset()
            # KONIEC JEDNEJ FAZY W EPOCE (TRENING LUB WALIDACJA)

        lr_scheduler.step()
        # KONIEC EPOKI


W trenowaniu klasyfikatorów typowo wykorzystywana jest **funkcja straty entropii krzyżowej** (klasa `nn.CrossEntropyLoss`) między estymowanym a prawdziwym rozkładem prawdopodobieństwa.
Jako argumenty obiektu `nn.CrossEntropyLoss` podajemy:
*   **nieznormalizowane wyjścia z sieci neuronowej (logity)** w formie tensora rozmiaru $(N, C)$, gdzie $N$ jest rozmiarem wsadu a $C$ liczbą klas
*   **identyfikatory prawdziwej klasy** w formie tensora rozmiaru $(N, )$zawierającego wartości z zakresu $0,\ldots,C-1$ (identyfikatory klas rozpoczynają się od zera).

**WAŻNE:** Ze względów numerycznych, do jako argument obiektu klasy `nn.CrossEntropyLoss` podajemy nieznormalizowane wyjście z sieci (logity) a NIE estymowane prawdopodobieństwo klas.

Niech $p$ będzie prawdziwym a $q$ estymowanym przez model rozkładem prawdopodobieństwa dyskretnej zmiennej losowej o wartościach $\mathcal{X} = \left\{ 0, \ldots, C-1 \right\}$ gdzie $C$ jest liczbą klas. **Entropia krzyżowa rozkładu $q$ względem $p$** jest dana jest wzorem:
$$
\mathcal{L}_{CE} = - \sum_{i \in \mathcal{X}} p_i \log q_i
$$

Dla pojedynczej próbki ze zbioru danych prawdziwy rozkład prawdopodobieństwa $p$ ma rozkład punktowy $p_k = 1$, gdzie $k$ jest identyfikatorem prawdziwej klasy. Wartość funkcji entropii krzyżowej upraszcza się zatem do postaci:
$$
\mathcal{l}_{CE} = - \log q_k \, ,
$$
gdzie $q_k$ jest estymowanym przez sieć prawdopodobieństwem prawdziwej klasy $k$.


In [ ]:
num_epochs = 8

# Utworzenie obiekty klasy nn.CrossEntropyLoss() obliczającego funkcję straty entropii krzyżowej
criterion = nn.CrossEntropyLoss()

# Utworzenie optymalizatora AdamW
optimizer = torch.optim.AdamW(classifier.parameters(), lr=1e-4, weight_decay=1e-5)

# Utworzenie planisty stopy uczenia
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-5)

In [ ]:
import wandb

# Logowanie do serwisu Weights&Biases monitorującego przebieg eksperymentów
wandb.login()

In [ ]:
run = wandb.init(project="MyExperiments")

train(classifier, dataloaders, criterion, optimizer, lr_scheduler, num_epochs)
run.finish()

Po zakończeniu trenowania modelu sprawdź przebiegi krzywych uczenia zalogowane w serwisie Weights&Biases: [link](https://wandb.ai/home).

Do zapisania i wczytania wag wytrenowanego modelu z dysku oraz zapisania stanu procesu uczenia modelu wykorzystywane są funkcje:

*   `torch.save` - zapisanie słownika stanu modelu na dysk.
*   `torch.load` - załadowanie słownika stanu modelu z dysku
*   `torch.nn.Module.load_state_dict` - ustawienie wag modułu sieci neuronowej z załadowanego słownika stanu modelu

Zwyczajowo słownik stanu modelu zapisywany jest w plikach z rozszerzeniem
`.pt` lub `.pth`.


**Słownik stanu modelu** oprócz wag sieci neuronowej zawiera aktualne parametry optymalizatora. Dzięki temu, po wczytaniu słownika stanu z sieci możliwa jest kontynuacja przerwanego treningu modelu.
Więcej informacji: [link](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

Zapisanie stanu modelu na dysku:
```
torch.save(model.state_dict(), PATH)
```
Wczytanie wag wytrenowanego modelu z dysku aby, wykorzystać go do inferencji:
```
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()
```
**Uwaga:** Przed wykorzystaniem modelu do inferencji należy pamiętać o przełączeniu go w tryb ewaluacji `model.eval()`. Jeśli model nie zostanie przełączony w tryb ewaluacji niektóre warstwy (dropout, batch norm) nie będą działały poprawnie.

In [ ]:
torch.save(classifier.state_dict(), 'my_model_weights.pth')

##Ewaluacja modelu

Finalna ewaluacja na zbiorze testowym + wszystkie metryki takie jak precyzja, odzysk per klasa, macierz pomyłek

In [ ]:
# Indeks elementu ze zbioru testowego
ndx = 2

x, target = datasets['test'][ndx]
print(f"Opinia: {test_dataset[ndx]['text']}")
print(f"Prawdziwa klasa: {target}")

x = x.to(device)

# BARDZO WAŻNE
classifier.eval()

with torch.inference_mode():
    # torch.inference_mode - opcjonalne, ale przyśpiesz wykonanie, bo nie jest tworzony graf obliczeń
    logits = classifier(x)

print(f"Wyjście z sieci (logity): {logits}")
probas = torch.nn.functional.softmax(logits, dim=-1)
print(f"Rozkład prawdopodobieństwa klas: {probas}")
print(f"Predykowana klasa: {logits.argmax(dim=-1)}")


Wyznacz wyniki predykcji dla wszystkich elementów zbioru testowego.
W tablicy `preds` zostanie zapisana predykowana klasa, a w tablicy
`targets` prawdziwa klasa każdego elementu.

In [ ]:
preds_l = []
targets_l = []

classifier.eval()

for X_batch, target in tqdm(dataloaders['test']):
    # Przenieś dane na odpowiednie urządzenie
    X_batch, target = X_batch.to(device), target.to(device)

    # Przejście w przód (forward)
    # Śledzenie historii obliczeń tylko w fazie trenowania
    with torch.inference_mode():
        logits = classifier(X_batch)
        _, preds = torch.max(logits, dim=-1)
        preds_l.extend(preds.cpu().numpy())
        targets_l.extend(target.cpu().numpy())

preds = np.array(preds_l)
targets = np.array(targets_l)

print(f"{preds.shape=}")
print(f"{targets.shape=}")

Do ewaluacji modelu wykorzystamy bibliotekę scikit-learn. Dla każdej klasy wyznaczymy metryki: precyzja (*precision*), czułość (*recall*), miara f1 (*f1-score*).

In [ ]:
from sklearn.metrics import classification_report

# Nazwy klas: 0=negatywna opinia, 1=pozytywna opinia
labels = ['negatywna', 'pozytywna']

report = classification_report(targets, preds, target_names = labels)
print(report)

Wyznaczenie macierzy pomyłek (*confusion matrix*). Wiersze macierzy pomyłek odpowiadają prawdziwej klasie, a kolumny predykowanej klasie.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(targets, preds)

# Create a heatmap
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Macierz pomyłek')
plt.xlabel('Predykowana klasa')
plt.ylabel('Prawdziwa klasa')
plt.show()


# Dodatkowe materiały

##Trening modelu z wykorzystaniem biblioteki PyTorch Lightning

PyTorch Lightning jest biblioteką pozwalającą na łatwiejszą implementację pętli treningowej w PyTorch, bez konieczności pisania powtarzalnych części kodu.
Wspiera trenowanie modeli w rozproszonym środowisku na wielu GPU oraz trening w trybie mieszanej precyzji (*mixed precision*).
Więcej informacji: [link](https://lightning.ai/docs/pytorch/stable/starter/introduction.html).

Poniżej znajduje się przykładowa implementacja pętli treningowej zdefiniowanej wcześniej sieci neuronowej klasy `SimpleNet` z wykorzystaniem biblioteki PyTorch Lightning.

In [ ]:
!pip install -q lightning

In [ ]:
from lightning.pytorch.loggers import WandbLogger

W pierwszym kroku należy opakować trenowany model w klasę dziedziczącą z klasy `lightning.LightningModule`. W tej klasie należy zdefiniować metody:
*   `training_step(...)` - jeden krok treningowy (przetworzenie jednego wsadu ze zbioru treningowego)
*   (opcjonalnie) `validation_step(...)` - jeden krok walidacyjny
*   (opcjonalnie) `test_step(...)` - jeden krok testowy
*  `configure_optimizers(...)` - konfiguracja optymalizatora i planisty stopy uczenia



In [ ]:
# define the LightningModule
import lightning as L


class LitNet(L.LightningModule):
    def __init__(self, classifier: nn.Module):
        super().__init__()
        self.classifier = classifier
        self.criterion = nn.CrossEntropyLoss()

        self.metric_train_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        self.metric_val_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        self.metric_test_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)

    def training_step(self, batch, batch_idx):
        # training_step implementuje jeden krok pętli treningowej
        x, target = batch
        logits = self.classifier(x)
        loss = self.criterion(logits, target)
        self.log("train/loss", loss, prog_bar=True)

        _, preds = torch.max(logits, dim=1)
        self.metric_train_acc(preds, target)
        self.log('train/accuracy', self.metric_train_acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, target = batch
        logits = self.classifier(x)
        loss = self.criterion(logits, target)
        self.log("val/loss", loss, prog_bar=True)

        _, preds = torch.max(logits, dim=1)
        self.metric_val_acc(preds, target)
        self.log('val/accuracy', self.metric_val_acc, on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, target = batch
        logits = self.classifier(x)

        _, preds = torch.max(logits, dim=1)
        self.metric_test_acc(preds, target)
        self.log('test/accuracy', self.metric_test_acc, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.classifier.parameters(), lr=1e-4, weight_decay=1e-5)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-5)
        return {"optimizer": optimizer,"lr_scheduler": lr_scheduler}

In [ ]:
# Init the network
classifier = SimpleNet(vocab_size, n_classes=2)
# Wrap in a lightning module
lit_model = LitNet(classifier)

Utworzenie obiektu klasy `lightning.Trainer` implementującego logikę pętli treningowej.

In [ ]:
wandb_logger = WandbLogger(project="MyExperiments")
trainer = L.Trainer(max_epochs=num_epochs, logger=wandb_logger)

Rozpoczęcie treningu modelu.

In [ ]:
trainer.fit(
    lit_model,
    train_dataloaders=dataloaders['train'],
    val_dataloaders=dataloaders['val']
    )

Po zakończeniu trenowania modelu sprawdź przebiegi krzywych uczenia zalogowane w serwisie Weights&Biases: [link](https://wandb.ai/home).

Ewaluacja wytrenowanego modelu na zbiorze testowym.

In [ ]:
trainer.test(ckpt_path="best", dataloaders=dataloaders['test'])

In [ ]:
# Zakończ logowanie danych eksperymentu
wandb_logger.experiment.finish()

In [ ]:
# Zwolnij pamięć
del classifier
del lit_model
del trainer

torch.cuda.empty_cache()

##Wykorzystanie statycznych wektorowych reprezentacji słów

Jako alternatywa do wektoryzacji metodą TD-IDF wykorzystamy statyczne wektorowe reprezentacje słów wyznaczone z wykorzystaniem metody Word2Vec. Skorzystamy z biblioteki Gensim ([link](https://radimrehurek.com/gensim/)).
Wektorową reprezentację dokumentu (opinii) wyznaczmy jako średnią z wektorowych reprezentacji słów w dokumencie. Podobnie jak w podejściu TD-IDF kolejność słów w dokumencie nie będzie uwzględniana.


In [ ]:
# Pobierz i rozpakuj pretrenowany model dla języka angielskiego: GoogleNews-vectors-negative300.bin.gz
!gdown 0B7XkCwpI5KDYNlNUTTlSS21pQmM
!gzip -d GoogleNews-vectors-negative300.bin.gz

In [ ]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess

# Załaduj model
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

Sprawdź wektorową reprezentację wybranego słowa.

In [ ]:
word_vector = model["car"]
print(f"Rozmiar wektorowej reprezentacji słowa: {word_vector.shape=}")
print(f"Pierwsze 20 elementów wektorowej reprezentacji: {word_vector[:20]=}")


In [ ]:
print(model)
#len(f"Liczba słów/n-gramów w słowniku: {model.words}")

Znajdź najbliższych sąsiadów podanego słowa, w sensie odległości kosinusowej, w przestrzeni reprezentacji.

In [ ]:
word = 'car'
print(model.most_similar(word, topn=5))

Wyznacz wektorową reprezentację dokumentu jako średnią z wektorowych reprezentacji słów w dokumencie.

In [ ]:
def compute_document_vector(document, model):
    tokens = simple_preprocess(document)
    vectors = []
    for word in tokens:
        # Pomijamy słowa których nie ma w słowniku
        if word in model:
            vectors.append(model[word])

    if len(vectors) == 0:
        document_vector =  np.zeros(model.vector_size).astype(np.float32)
    else:
        document_vector = np.mean(vectors, axis=0).astype(np.float32)

    return document_vector

In [ ]:
ndx = 101
data_item = train_dataset[ndx]
print(data_item)
doc_embedding = compute_document_vector(data_item['text'], model)
print(f"")
print(f"Rozmiar reprezentacji dokumentu: {doc_embedding.shape}")

Wyzncz wektorowe reprezentacje wszystkich dokumentów w każdym ze zbiorów danych (treningowym, walidacyjnym i testowym).

In [ ]:
from torch.utils.data import TensorDataset


def make_dataset(hf_dataset):
    doc_vectors = []
    labels = []
    for item in hf_dataset:
        doc_vectors.append(compute_document_vector(item['text'], model))
        labels.append(item['label'])
    return TensorDataset(torch.tensor(doc_vectors), torch.tensor(labels))


# Utwórz trzy zbiory danych: treningowy, walidacyjny i testowy
datasets = {
    'train': make_dataset(train_dataset),
    'val': make_dataset(val_dataset),
    'test': make_dataset(test_dataset)
}

# Wyświetl przykładowy element
print(datasets['train'][0])

Utwórz ładowarki danych (*Dataloaders*) dla każdego zbioru danych.

In [ ]:
batch_size = 128

dataloaders = {split: DataLoader(datasets[split], batch_size=batch_size, shuffle=split=='train', num_workers=0) for split in datasets}

Wytrenujemy model klasyfikatora sentymentu tekstu w oparciu o wektorową reprezentację dokumentów. Wykorzystamy zaimplementowany wcześniej model wielowarstwowego perceptronu `SimpleNet`. Rozmiar wejścia będzie równy rozmiarowi wektorowej reprezentacji (`model.vector_size`) a rozmiar wyjścia równy 2 (dwie klasy: sentyment negatywny i pozytywny).

Trening modelu przeprowadzimy z wykorzystaniem biblioteki PyTorch Lightning. W tym celu wykorzystamy utworzoną wcześniej klasę `LitNet` dziedziczącą z klasy `lightning.LightningModule`.

In [ ]:
# Init the network
classifier = SimpleNet(model.vector_size, n_classes=2)
# Wrap in a lightning module
lit_model = LitNet(classifier)

In [ ]:
wandb_logger = WandbLogger(project="MyExperiments")
trainer = L.Trainer(max_epochs=num_epochs, logger=wandb_logger)

In [ ]:
trainer.fit(
    lit_model,
    train_dataloaders=dataloaders['train'],
    val_dataloaders=dataloaders['val']
    )

In [ ]:
trainer.test(ckpt_path="best", dataloaders=dataloaders['test'])